### Import TensorFlow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.models import Model
import numpy as np 
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Download and prepare the CIFAR10 dataset


The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.


In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    # plt.xlabel(class_names[train_labels[i][0]])
    plt.xlabel(train_labels[i][0])
plt.show()

### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument `input_shape` to our first layer.



In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(10, activation='softmax'))

In [0]:
model.summary()

### 算AP mAP

In [0]:
# 取出sigmoid層的輸出，並建立model
dense_layer_model = Model(inputs = model.input ,
                           outputs = model.get_layer('dense').output)
# 以此model的預測值作為輸出
dense_output = dense_layer_model.predict(train_images)

dense_output_test = dense_layer_model.predict(test_images)

# # 50000 * 64
# dense_output.shape
# # (10000, 64)
# dense_output_test.shape

# 將dense_output轉成np array，因為可以函式去處理01情況
dense_nparray = np.array(dense_output)
dense_nparray_test = np.array(dense_output_test)

# 裡面的值若>=0.5 就輸出1 反之輸出0
dense_nparray_binary = np.where(dense_nparray >= 0.5 , 1 , 0)
dense_nparray_test_binary = np.where(dense_nparray_test >= 0.5 , 1 , 0)
# 取前50筆
train_b = dense_nparray_binary
test_b = dense_nparray_test_binary[:5000]

test_b1 = test_b.T
test_b2 = (1-test_b).T

# 每1筆test image對所有train image算出hamming : (50000, 5000)
logprobs = np.dot(train_b, test_b1) + np.dot((1-train_b), test_b2)

# 高到低的index
logprobs1 = (-logprobs).argsort()
# log_sort =-np.sort(-logprobs)

# 將排好的index做轉置，以便可以直接當成每個train_label的新索引
logprobs1 = logprobs1.T
# print(logprobs1.shape) output : (50000, 5000)

# 此為依照ham_dist排好的新training label
train_label_ordered = train_labels[logprobs1].reshape(50000 , 5000)

test = test_labels[:5000].reshape(5000)

In [0]:
average_precision=[]
for i in range (len(train_label_ordered) ):
  presicion = 0
  fraction = 0
  for j in range (len(test)):
    if train_label_ordered[i][j] == test[j] : 
      fraction = fraction + 1
      presicion = presicion + (fraction / (j+1) )
  # AP
  presicion = presicion / 5000
  average_precision.append(presicion)

In [0]:
mAP = np.mean(average_precision)
mAP

### Compile and train the model

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Evaluate the model

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [0]:
print(test_acc)